<a href="https://colab.research.google.com/github/anyelyra/Mid-Term-Task/blob/main/Mid-Term-Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama: Anyelyra Kantata

NPM: 2206048625



# Import Library yang Dibutuhkan

In [1]:
!pip install transformers
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

import numpy as np
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset, TensorDataset, random_split
import torch.nn.functional as F
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


`Transformers`: Library yang menyediakan berbagai model transformer pre-trained, termasuk BERT, yang bisa digunakan untuk tugas NLP seperti klasifikasi teks.

`PyTorch`: Framework deep learning yang digunakan untuk memodelkan neural network dalam model BERT.

`sklearn`: Menyediakan metrik evaluasi.

# Persiapkan Dataset

Kita akan membuat dataset contoh, dengan beberapa teks dan label (1 untuk teks positif, 0 untuk negatif). Setelah itu, kita tokenisasi teks dan ubah menjadi tensor agar bisa diolah oleh model BERT.

In [2]:
data = [
    ("I feel happy today!", 1),
    ("This is a sad story.", 0),
    ("Everything is great in my life.", 1),
    ("I am feeling terrible right now.", 0),
    ("I am so excited about the concert!", 1),
    ("Today was a boring day.", 0),
    ("What a wonderful surprise!", 1),
    ("That was an awful experience.", 0),
    ("The movie was amazing!", 1),
    ("It was a disappointing match.", 0),
    ("I feel joyful after hearing the news.", 1),
    ("The outcome was dismal.", 0),
    ("I am thrilled to start my new job!", 1),
    ("This food tastes bad.", 0),
    ("I'm delighted to meet you.", 1),
    ("This place looks dreadful.", 0),
    ("What a splendid idea!", 1),
    ("It was a regrettable decision.", 0),
    ("I have a fantastic feeling about this!", 1),
    ("The report was lackluster.", 0),
    ("I am very optimistic about the future.", 1),
    ("It was an irritating situation.", 0),
    ("Feeling great after a long workout!", 1),
    ("This weather is not pleasant.", 0),
    ("I'm so grateful for my friends!", 1),
    ("The service was unacceptable.", 0),
    ("I feel rejuvenated after my vacation.", 1),
    ("That was a horrible mistake.", 0),
    ("I am looking forward to the weekend!", 1),
    ("I cannot stand this traffic.", 0),
    ("It's such a lovely day outside.", 1),
    ("The lecture was uninteresting.", 0),
    ("I'm ecstatic about the upcoming event!", 1),
    ("That was a miserable experience.", 0),
    ("I feel so grateful for everything I have.", 1),
    ("It was a catastrophic failure.", 0),
    ("I'm excited to travel again.", 1),
    ("This project is going poorly.", 0),
    ("I am so proud of my accomplishments.", 1),
    ("That was a lackluster performance.", 0),
    ("I feel confident about my skills.", 1),
    ("This product is disappointing.", 0),
    ("I am overjoyed with the results!", 1),
    ("The response was underwhelming.", 0),
    ("I feel so inspired right now.", 1),
    ("It was a disheartening situation.", 0),
    ("What an encouraging atmosphere!", 1),
    ("I feel trapped in this situation.", 0),
    ("Today has been so refreshing!", 1),
    ("This is a terrible plan.", 0),
    ("I am really pumped for the game!", 1),
    ("That was a discouraging comment.", 0),
    ("I feel a sense of peace today.", 1),
    ("That was a stressful day.", 0),
    ("I had a fantastic day!", 1),
    ("This is the worst experience ever.", 0),
    ("I'm feeling incredibly positive today!", 1),
    ("That was a disappointing outcome.", 0),
    ("I'm thrilled with the results!", 1),
    ("This situation is quite frustrating.", 0),
    ("I love spending time with my family!", 1),
    ("I can't believe how bad this is.", 0),
    ("What a great surprise!", 1),
    ("This product is not what I expected.", 0),
    ("I'm so happy to be here!", 1),
    ("The food was awful.", 0),
    ("I can't wait for the concert!", 1),
    ("This has been a terrible week.", 0),
    ("I'm so grateful for my health!", 1),
    ("I am unhappy with my purchase.", 0),
    ("Today is a wonderful day!", 1),
    ("I feel so down right now.", 0),
    ("I'm excited about the future!", 1),
    ("That was a disappointing performance.", 0),
    ("I'm feeling motivated and inspired!", 1),
    ("I hate this weather.", 0),
    ("What an amazing opportunity!", 1),
    ("I'm frustrated with the service.", 0),
    ("I feel blessed to have such great friends.", 1),
    ("This event was poorly organized.", 0),
    ("I'm looking forward to the holidays!", 1),
    ("This is a horrible situation.", 0),
    ("I feel fulfilled after finishing my project!", 1),
    ("That was a dull movie.", 0),
    ("I'm thrilled to learn something new!", 1),
    ("That was a big mistake.", 0),
    ("I'm feeling so hopeful today!", 1),
    ("This is a major setback.", 0),
    ("I'm enjoying every moment!", 1),
    ("I feel completely let down.", 0),
    ("I'm ecstatic about the results!", 1),
    ("This has been an exhausting day.", 0),
    ("I'm so proud of what I've accomplished!", 1),
    ("That was a terrible decision.", 0),
    ("I'm feeling great about my progress!", 1),
    ("This is such a letdown.", 0),
    ("I'm so happy with my choices!", 1),
    ("This project is a disaster.", 0),
    ("What a delightful surprise!", 1),
    ("I'm tired of this constant negativity.", 0),
    ("I feel like celebrating!", 1),
    ("That was a very sad story.", 0),
    ("I feel so grateful for my life!", 1),
    ("This was a frustrating experience.", 0),
    ("I'm ready to take on the world!", 1),
    ("That was a waste of time.", 0),
]

# Konversi Data menjadi TensorDataset

In [3]:
texts, labels = zip(*data)

class CustomDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

dataset = CustomDataset(texts, labels)

# Membagi Data menjadi Latih dan Validasi

In [4]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Membuat BERT Tokenizer

Kita perlu membuat `DataLoader` agar data bisa diproses dalam batch selama pelatihan.


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(dataset):
    input_ids = []
    attention_masks = []
    labels = []

    for text, label in dataset:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])
        labels.append(label)

    return torch.cat(input_ids), torch.cat(attention_masks), torch.tensor(labels)

train_input_ids, train_attention_masks, train_labels = tokenize_data(train_dataset)
val_input_ids, val_attention_masks, val_labels = tokenize_data(val_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Membuat DataLoaders untuk Latih dan Validasi



In [6]:
train_data = TensorDataset(train_input_ids, train_attention_masks, train_labels)
val_data = TensorDataset(val_input_ids, val_attention_masks, val_labels)

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8, shuffle=False)

# Model

In [7]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Set the model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3

# Set a learning rate scheduler to help with fine-tuning
from transformers import get_scheduler

num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# Model Training





In [8]:
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

    # Validation phase
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, dim=1).tolist())
            true_labels.extend(labels.tolist())

    # Calculate metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)

    print(f'Epoch: {epoch + 1}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

100%|██████████| 11/11 [00:04<00:00,  2.39it/s]


Epoch: 1, Accuracy: 0.9091, Precision: 1.0000, Recall: 0.8000, F1 Score: 0.8889


100%|██████████| 11/11 [00:01<00:00,  5.60it/s]


Epoch: 2, Accuracy: 0.9545, Precision: 0.9091, Recall: 1.0000, F1 Score: 0.9524


100%|██████████| 11/11 [00:01<00:00,  5.72it/s]


Epoch: 3, Accuracy: 0.9545, Precision: 0.9091, Recall: 1.0000, F1 Score: 0.9524


# Metrik Evaluasi Keseluruhan

In [10]:
overall_accuracy = accuracy_score(true_labels, predictions)
overall_precision = precision_score(true_labels, predictions)
overall_recall = recall_score(true_labels, predictions)
overall_f1 = f1_score(true_labels, predictions)

In [11]:
print(f'Overall Accuracy: {overall_accuracy:.4f}')
print(f'Overall Precision: {overall_precision:.4f}')
print(f'Overall Recall: {overall_recall:.4f}')
print(f'Overall F1 Score: {overall_f1:.4f}')

Overall Accuracy: 0.9545
Overall Precision: 0.9091
Overall Recall: 1.0000
Overall F1 Score: 0.9524
